**Coursebook: SQL and Data Visualization with Pandas**
- Part 4 of Data Analytics Specialization
- Course Length: 12 hours
- Last Updated: April 2019
___

- Author: [Samuel Chan](https://github.com/onlyphantom)
- Developed by [Algoritma](https://algorit.ma)'s product division and instructors team

# Background

## Top-Down Approach 

The coursebook is part 4 of the **Data Analytics Specialization** offered by [Algoritma](https://algorit.ma). It takes a more accessible approach compared to Algoritma's core educational products, by getting participants to overcome the "how" barrier first, rather than a detailed breakdown of the "why". 

This translates to an overall easier learning curve, one where the reader is prompted to write short snippets of code in frequent intervals, before being offered an explanation on the underlying theoretical frameworks. Instead of mastering the syntactic design of the Python programming language, then moving into data structures, and then the `pandas` library, and then the mathematical details in an imputation algorithm, and its code implementation; we would do the opposite: Implement the imputation, then a succinct explanation of why it works and applicational considerations (what to look out for, what are assumptions it made, when _not_ to use it etc).

## Learn-by-Building

This coursebook is intended for participants who have completed the preceding courses offered in the **Data Analytics Developer Specialization**. This is the fourth course, **SQL and Data Sources**.

The coursebook focuses on:
- Querying from SQL Databases
- Database Joins
- Working with JSON
- Accessing open API

At the end of this course is a Graded Asssignment section, where you are expected to apply all that you've learned on a new dataset, and attempt the given questions.

In [4]:
import pandas as pd
books = pd.read_csv('dataanalysis/data_input/books_c.csv')
books.head()

bookID                                              title       authors  \
0       1  Harry Potter and the Half-Blood Prince (Harry ...  J.K. Rowling   
1       2  Harry Potter and the Order of the Phoenix (Har...  J.K. Rowling   
2       3  Harry Potter and the Sorcerer's Stone (Harry P...  J.K. Rowling   
3       4  Harry Potter and the Chamber of Secrets (Harry...  J.K. Rowling   
4       5  Harry Potter and the Prisoner of Azkaban (Harr...  J.K. Rowling   

   average_rating        isbn         isbn13 language_code  # num_pages  \
0            4.56  0439785960  9780439785969           eng          652   
1            4.49  0439358078  9780439358071           eng          870   
2            4.47  0439554934  9780439554930           eng          320   
3            4.41  0439554896  9780439554893           eng          352   
4            4.55  043965548X  9780439655484           eng          435   

   ratings_count  text_reviews_count  
0        1944099               26249  
1        1996446               27613  
2        5629932               70390  
3           6267                 272  
4        2149872               33964

# Working with SQL Databases

There are a great number of python modules that provide functionalities to work with databases of all variants and flavors. For a MySQL database, we may form a connection using `pymysql` or one of many other alternatives:

```
import pymysql
conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    db=database)
```

We can then use `pd.read_sql_query()`, passing in the connection:
```
sales = pd.read_sql_query("SELECT * FROM sales", conn)
```

Under the hood, `pandas` uses SQLAlchemy so any database supported by that library will work. This isn't something you need to worry about at this stage of your learning journey, but for the sake for practice, let's also see how a connection URI for a SQLite database looks like:

In [12]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data_input/chinook.db")

albums = pd.read_sql_query("SELECT * FROM albums", conn)
albums.head()

AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3

In the above command, we asked for all columns of a table to be returned to us through the `SELECT *` command. Well, columns of which table? That would be `tables`. Together they form an SQL query:

`SELECT * FROM albums`

The database we're working with have a few tables populated with sample data. The database has the following schema:
![](assets/chinookschema.png)

#### Knowledge Check

We'll create a `DataFrame`: this time select all columns from the `artists` table. Recall that when we use `pd.read_sql_query()` command we pass in the SQL query as a string, and add a connection as the second parameter. Save the output as a `DataFrame`.

Your DataFrame should be constructed like this:

`__ = pd.read_sql_query("SELECT __ FROM __ ", conn)`

Question:
1. How many rows are there in your DataFrame?

In [13]:
## Your code below


## -- Solution code

The `pd.read_sql_query` is most commonly used with that two parameters above, but on its [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) is a list of other parameters we can use as well. 

In the following cell, we use a similar SQL query with an additional `LIMIT` statement to limit the output to the first 5 records (rows). However, notice that we also set `index_col` so the specified column is recognized as the index:

In [19]:
pd.read_sql_query("SELECT * FROM artists LIMIT 5", 
                  conn, 
                  index_col='ArtistId')

Name
ArtistId                   
1                     AC/DC
2                    Accept
3                 Aerosmith
4         Alanis Morissette
5           Alice In Chains

## SQL Joins

`JOIN` statements are used to combine records from two tables. We can have as many `JOIN` operations as we want in a SQL query.

Below is a diagram of the different types of SQL `JOIN` operations:

![](assets/sqljoins.png)

Credit: Data & Object Factory, LLC

In most business scenarios though, a `LEFT JOIN` is almost always the type of `JOIN` you want - it is very direct (and therefore easy to reason about). Left join return all records in the left table regardless if any of those records have a match in the right table.

The `INNER JOIN` is also very intuitive and easily understood. This query return all of the records in the left table that has a matching record in the right table.

> As a personal side note, I've worked at companies where `RIGHT JOIN` is outright forbidden in favor of `LEFT JOIN`: directness and ease-of-understanding aside, all right joins can be replaced by the opposite left join. 
> 
> The `OUTER JOIN` (also referred to as `FULL JOIN`) is also quite uncommon in practice. Performance reason aside, an outer join return all of the records from both tables regardless if there is a match or not, resulting in a DataFrame that has potentially a lot of `NULL` values.

Consider the database schema illustraation again and pay attention to two tables and their respective columns:

1. `albums`: 
    - `AlbumId`, `Title`, `ArtistId`

2. `artists`: `
    - `ArtistId`, `Name`    

We want a `pandas` DataFrame containing the `AlbumId`, `Title` and `Name`. Notice that `Name` is from the `artists` table while the other columns are from the `albums` table. What is a reasonable strategy?

The most straightforward solution is the `LEFT JOIN`, let's see an example:

In [123]:
albums = pd.read_sql_query("SELECT AlbumId, Title, a.Name \
                           FROM albums \
                           LEFT JOIN artists as a \
                           ON a.ArtistId = albums.ArtistId", conn)
albums.head()

AlbumId                                  Title       Name
0        1  For Those About To Rock We Salute You      AC/DC
1        2                      Balls to the Wall     Accept
2        3                      Restless and Wild     Accept
3        4                      Let There Be Rock      AC/DC
4        5                               Big Ones  Aerosmith

Notice that in the code above, we place a backslash (`\`) character so we have line continuation and the newline will be ignored. This allows SQL to treat the entire query string as if they were essentially one line.

In [35]:
pd.read_sql_query("SELECT * FROM albums", conn).head()

AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3

#### Knowledge Check

Consider the database schema illustraation again and pay attention to two tables and their respective columns:

1. `albums`: `AlbumId`, `Title`, `ArtistId`

2. `tracks`: `TrackId`, `Name`, `AlbumId`, `GenreId`, ... `UnitPrice` 
    
3. `genres`: `GenreId`, `Name`

Create a `DataFrame` containing all columns from the `tracks` table; Additionally, it should also contain:
    - The `Title` column from the `albums` table
    - The `Name` column from the `artists` table
    - The `Name` column from the `genres` table 

> **Hint 1**: In your `SELECT` statement, you can use `SELECT tracks.* FROM TRACKS` to select all columns from the `TRACKS` table
> 
> **Hint 2**: When we write `SELECT tracks.Name as tracksName`, we are renaming the output column from `Name` to `tracksName` using a technique called column aliasing. You may optionally consider doing this for columns that share the same name across different tables 

Set the `TrackId`column to be the index. The resulting `DataFrame` should has 11 columns.

Give your `DataFrame` a name: name it `tracks`. Perform EDA on `tracks` to answer the following question:

1. Use `tail()` to inspect the last 5 rows of data. Which genre is present in the last 5 rows of our `tracks` DataFrame (Check all that apply)?
    - [ ] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

2. Apply `pd.crosstab(..., columns='count')`, `.value_counts()`, or any other techniques you've learned to compute the frequency table of Genres in your DataFrame. Which is among the top 3 most represented genres in the `tracks` DataFrame?
    - [ ] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

3. Use `groupby()` on Artist Name and compute the `mean()` on the `UnitPrice` of each tracks. You will realize that most artists price their tracks at 0.99 (`mean`) but there are several artists where the `mean()` is 1.99. Which of the Artist has a mean of 0.99 `UnitPrice`:
    - [ ] The Office
    - [ ] Aquaman
    - [ ] Pearl Jam
    - [ ] Lost

In [25]:
## Your code below


## -- Solution code

In [8]:
aapl_fb = stock.loc[:,(['Open', 'Close', 'Adj Close'],['AAPL', 'FB'])]
aapl_fb.head()

Attributes        Open                   Close               Adj Close  \
Symbols           AAPL          FB        AAPL          FB        AAPL   
Date                                                                     
2018-01-02  170.160004  177.679993  172.259995  181.419998  168.987320   
2018-01-03  172.529999  181.880005  172.229996  184.669998  168.957886   
2018-01-04  172.539993  184.899994  173.029999  184.330002  169.742706   
2018-01-05  173.440002  185.589996  175.000000  186.850006  171.675278   
2018-01-08  174.350006  187.199997  174.350006  188.279999  171.037628   

Attributes              
Symbols             FB  
Date                    
2018-01-02  181.419998  
2018-01-03  184.669998  
2018-01-04  184.330002  
2018-01-05  186.850006  
2018-01-08  188.279999

## WHERE statements

We've seen how to use do some of the most common SQL operations this far. In particular, we have:

- Learned how to write `SELECT` statements  
- Use `index_col` in the `pd.read_sql_query()` method  
- SQL Join operations
- Use SQL Aliases

In the following example, we'll look at one more technique in the SQL arsenal: the `WHERE` clause

A `WHERE` clause is followed by a **condition**. If we want to query for all invoices where country of the billing address is Germany, we can add a `Where` clause to our sql query string:

In [98]:
germany = pd.read_sql_query("SELECT * FROM invoices WHERE BillingCountry = 'Germany'", conn)
germany.head()

InvoiceId  CustomerId          InvoiceDate           BillingAddress  \
0          1           2  2009-01-01 00:00:00  Theodor-Heuss-Straße 34   
1          6          37  2009-01-19 00:00:00         Berger Straße 10   
2          7          38  2009-02-01 00:00:00      Barbarossastraße 19   
3         12           2  2009-02-11 00:00:00  Theodor-Heuss-Straße 34   
4         29          36  2009-05-05 00:00:00       Tauentzienstraße 8   

  BillingCity BillingState BillingCountry BillingPostalCode  Total  
0   Stuttgart         None        Germany             70174   1.98  
1   Frankfurt         None        Germany             60316   0.99  
2      Berlin         None        Germany             10779   1.98  
3   Stuttgart         None        Germany             70174  13.86  
4      Berlin         None        Germany             10789   1.98

`WHERE` conditions can be combined with `AND`, `OR` and `NOT`. Supposed we want to create a DataFrame containing all invoices where the billing country is **not** Indonesia, we can do the following:

In [105]:
international = pd.read_sql_query("SELECT * FROM invoices WHERE NOT BillingCountry = 'Indonesia'", conn)
international.head()

InvoiceId  CustomerId          InvoiceDate           BillingAddress  \
0          1           2  2009-01-01 00:00:00  Theodor-Heuss-Straße 34   
1          2           4  2009-01-02 00:00:00         Ullevålsveien 14   
2          3           8  2009-01-03 00:00:00          Grétrystraat 63   
3          4          14  2009-01-06 00:00:00           8210 111 ST NW   
4          5          23  2009-01-11 00:00:00          69 Salem Street   

  BillingCity BillingState BillingCountry BillingPostalCode  Total  
0   Stuttgart         None        Germany             70174   1.98  
1        Oslo         None         Norway              0171   3.96  
2    Brussels         None        Belgium              1000   5.94  
3    Edmonton           AB         Canada           T6G 2C7   8.91  
4      Boston           MA            USA              2113  13.86

#### Knowledge Check

Edit the following code to include a `WHERE` clause. We want the returned DataFrame to contain only the `Pop` genre and only when the `UnitPrice` of the track is 0.99:

```
popmusic = pd.read_sql_query("SELECT tracks.*, genres.Name as GenreName \
                            FROM tracks \
                            LEFT JOIN genres ON _____ \
                            WHERE genres.Name = ____ AND _____, 
                           conn,
                           index_col='TrackId'

)
```

Question:
1. How many rows are there in `popmusic`?

In [106]:
## Your code below


## -- Solution code

# Fetching Data from API

In Python, there are a lot of API Wrapper that can be easily used to access data from various API. Application programming interface, are created to provide a certain kind of "request" to a server. `pandas_datareader` we used in previous week is one of the example of API wrapper. Head to this [link](https://github.com/realpython/list-of-python-api-wrappers) to see a compilation of the most popular open ones. If you're working in a company, ask your IT division whether or not the data you need can be provided using an API.



## Working with JSON

Let's try to access Jakarta Open API. You can access the website [here](http://api.jakarta.go.id/). Some of APIs implement an authorization procedure to help them maintain data security and cost. In this case, Jakarta Open API also requires you to sign up to their portal and register an application before providing us with a key access. For this course, we are going to use a pre-generated key. Do note that to start developing using this API you need to create your own key.

An HTTP request commonly include 4 main components: URL, headers, parameters, and body. While providing a URL is a must, header, parameter, and body is not required to create a request. Based on the [official documentation](http://api.jakarta.go.id/endpoint) of the API we need to provide `Authorization` as one of the header and specify our key access:

In [115]:
import requests as r

url = "http://api.jakarta.go.id/v1/rumahsakitumum/"
key = "oDl6JQKN1uyovOIi+X1sv6uYSrfn+a4HfH5OS5Sw7hIAa/dOh7I5umOUDz8L76xk"

response = r.get(url,
                headers={
                    "Authorization": key
                })

data = response.json()

The request response can have multiple components. JSON is a common data types to be exchanged between servers and clients. The simple why to that is because it is basically a structured set of string that has particular structure to it. In Python, it is similar to an object called dictionary. To see what kind of data is fetched by the API, let's inspect the json keys:

In [93]:
data.keys()

dict_keys(['status', 'count', 'data'])

Looking at the key id of each dictionary, let's access the `count` key to find out how much data we are fetching:

In [96]:
data['count']

112

A total of 112 general hospital seems to be fetched from the API. If you print out the `data` key content (try `data['data']`), you will find a set of list as long as the data within `count` key:

In [100]:
len(data['data'])

112

> Note: Each of API provided different schema of information. In this particular case we acquired a total of 3 keys: status, count, and data. Don't expect to access different API and received a same set of JSON schema. To be able to understand what kind of data we are going to receive from the API, the best way is to just ask of API documentation provided by the developer.

Now since we know that our information is stored within `data` key, let's transform it to `pandas` data frame:

In [117]:
hospital = pd.io.json.json_normalize(data['data'])
hospital.head()

email              faximile  \
0           kusmedi@gmail.com, rsd_tarakan@yahoo.com   [3503412,  3863309]   
1                          rsalmintohardjo@ymail.com             [5711997]   
2                             information@ssmh.co.id    [5705505, 5727213]   
3               tedjowa@yahoo.com, mail@rscikini.com  [31924663, 31908391]   
4  panela_ramadita@yahoo.com, info@rsmentengmitra...             [3146309]   

   id                             jenis_rsu  kode_kecamatan  kode_kelurahan  \
0   2               Rumah Sakit Umum Daerah         3173080      3173080001   
1   3  Rumah Sakit Umum TNI - Angkatan Laut         3173010      3173010002   
2   4                      Rumah Sakit Umum         3173010      3173010003   
3   5                      Rumah Sakit Umum         3173020      3173020003   
4   6                      Rumah Sakit Umum         3173020      3173020003   

   kode_kota  kode_pos  latitude                   location.alamat  \
0       3173     10150 -6.171333           Jl. Kyai Caringin No. 7   
1       3173     10210 -6.210848        Jl. Bendungan Hilir No. 17   
2       3173     10220 -6.209445  Jl. Jenderal Sudirman Kavling 86   
3       3173     10330 -6.191270           Jl. Raden Saleh No. 40    
4       3173     10340 -6.187353             Jl. Kali Pasir  No. 9   

   location.latitude  location.longitude   longitude            nama_rsu  \
0          -6.171333          106.810013  106.810013             Tarakan   
1          -6.210848          106.811996  106.811996     DR. Mintohardjo   
2          -6.209445          106.819382  106.819382      Sahid Sahirman   
3          -6.191270          106.841408  106.841408          PGI Cikini   
4          -6.187353          106.839081  106.839081  Menteng Mitra Afia   

                       telepon                     website  
0  [3503150, 3503003, 3508993]    www.rstarakanjakarta.com  
1               [5703081 - 85]   www.rsaldrmintohardjo.com  
2           [5704591, 5703231]              www.ssmh.co.id  
3                   [38997777]            www.rscikini.com  
4                    [3154050]  www.rsmentengmitraafia.com

#### Knowledge Check

We already know that JSON consist of several key, and each keys will have a content. The content, however, does not constrained to follow a certain schema. This can introduces a fundamentally different data structure than a simple tabular data. Data frame consist of a same-length series that each element within the series hold 1 value. Which of the following column retrieved from our data that seems to hold multiple values from one column:

- [ ] jenis_rsu  
- [ ] faxmile
- [ ] email
- [ ] kode_kecamatan
- [ ] telepon  

## DataFrame Joins

Similar to relational database. Sometimes, we  need to access multiple data sources before producing the right shape of data we wanted. In the previous chapter, we have retrieved information of general hospitals within Jakarta. Notice how the kode_kecamatan is stored under a specific code. To be able to map it to the true district name we can try to do the lookup technique by joining 2 data frames.

In [119]:
url = "http://api.jakarta.go.id/v1/kecamatan/?format=geojson"

response = r.get(url,
                headers={
                    "Authorization": "oDl6JQKN1uyovOIi+X1sv6uYSrfn+a4HfH5OS5Sw7hIAa/dOh7I5umOUDz8L76xk"
                })

data = response.json()
data.keys()

dict_keys(['type', 'features'])

Notice how the keys are different with the previous JSON we retrieved. Let's try to illustrate some trial and error process in understanding the schema:

In [120]:
pd.DataFrame.from_dict(data['features']).head()

geometry  \
0  {'type': 'Polygon', 'coordinates': [[[106.9140...   
1  {'type': 'Polygon', 'coordinates': [[[106.8487...   
2  {'type': 'Polygon', 'coordinates': [[[106.8575...   
3  {'type': 'Polygon', 'coordinates': [[[106.8840...   
4  {'type': 'Polygon', 'coordinates': [[[106.9091...   

                                          properties     type  
0  {'kode_provinsi': 31, 'nama_provinsi': 'DAERAH...  Feature  
1  {'kode_provinsi': 31, 'nama_provinsi': 'DAERAH...  Feature  
2  {'kode_provinsi': 31, 'nama_provinsi': 'DAERAH...  Feature  
3  {'kode_provinsi': 31, 'nama_provinsi': 'DAERAH...  Feature  
4  {'kode_provinsi': 31, 'nama_provinsi': 'DAERAH...  Feature

The data seems to retrieved 2 main information: geometry and properties. Since the main interest is retrieving the code and name of district table for lookup reference, let's try to extract the properties column. Now since this is a multiple level JSON, the `json_normalize` function we used earlier will conveniently transform the multilevel as new columns:

In [123]:
district  = pd.io.json.json_normalize(data['features']).filter(regex='properties')
district.head()

properties.kode_kecamatan  properties.kode_kota  properties.kode_provinsi  \
0                    3172040                  3172                        31   
1                    3171010                  3171                        31   
2                    3172010                  3172                        31   
3                    3172020                  3172                        31   
4                    3172030                  3172                        31   

  properties.nama_kecamatan properties.nama_kota  \
0                   Makasar        Jakarta Timur   
1                 Jagakarsa      Jakarta Selatan   
2                Pasar Rebo        Jakarta Timur   
3                   Ciracas        Jakarta Timur   
4                  Cipayung        Jakarta Timur   

        properties.nama_provinsi  
0  DAERAH KHUSUS IBUKOTA JAKARTA  
1  DAERAH KHUSUS IBUKOTA JAKARTA  
2  DAERAH KHUSUS IBUKOTA JAKARTA  
3  DAERAH KHUSUS IBUKOTA JAKARTA  
4  DAERAH KHUSUS IBUKOTA JAKARTA

Now since the lookup table has been prepared, we now need to join the 2 data frames into one. We are going to be using `hospital` as our main table and `district` as the lookup table. To merge 2 of them using the technique we have learned in SQL section, we can use `pd.merge`:

In [124]:
df_merged = pd.merge(hospital, 
                     district, 
                     how='left', 
                     left_on='kode_kecamatan',
                     right_on='properties.kode_kecamatan')
df_merged.head()

email              faximile  \
0           kusmedi@gmail.com, rsd_tarakan@yahoo.com   [3503412,  3863309]   
1                          rsalmintohardjo@ymail.com             [5711997]   
2                             information@ssmh.co.id    [5705505, 5727213]   
3               tedjowa@yahoo.com, mail@rscikini.com  [31924663, 31908391]   
4  panela_ramadita@yahoo.com, info@rsmentengmitra...             [3146309]   

   id                             jenis_rsu  kode_kecamatan  kode_kelurahan  \
0   2               Rumah Sakit Umum Daerah         3173080      3173080001   
1   3  Rumah Sakit Umum TNI - Angkatan Laut         3173010      3173010002   
2   4                      Rumah Sakit Umum         3173010      3173010003   
3   5                      Rumah Sakit Umum         3173020      3173020003   
4   6                      Rumah Sakit Umum         3173020      3173020003   

   kode_kota  kode_pos  latitude                   location.alamat  ...  \
0       3173     10150 -6.171333           Jl. Kyai Caringin No. 7  ...   
1       3173     10210 -6.210848        Jl. Bendungan Hilir No. 17  ...   
2       3173     10220 -6.209445  Jl. Jenderal Sudirman Kavling 86  ...   
3       3173     10330 -6.191270           Jl. Raden Saleh No. 40   ...   
4       3173     10340 -6.187353             Jl. Kali Pasir  No. 9  ...   

    longitude            nama_rsu                      telepon  \
0  106.810013             Tarakan  [3503150, 3503003, 3508993]   
1  106.811996     DR. Mintohardjo               [5703081 - 85]   
2  106.819382      Sahid Sahirman           [5704591, 5703231]   
3  106.841408          PGI Cikini                   [38997777]   
4  106.839081  Menteng Mitra Afia                    [3154050]   

                      website properties.kode_kecamatan properties.kode_kota  \
0    www.rstarakanjakarta.com                 3173080.0               3173.0   
1   www.rsaldrmintohardjo.com                 3173010.0               3173.0   
2              www.ssmh.co.id                 3173010.0               3173.0   
3            www.rscikini.com                 3173020.0               3173.0   
4  www.rsmentengmitraafia.com                 3173020.0               3173.0   

   properties.kode_provinsi  properties.nama_kecamatan  properties.nama_kota  \
0                      31.0                     Gambir         Jakarta Pusat   
1                      31.0                Tanah Abang         Jakarta Pusat   
2                      31.0                Tanah Abang         Jakarta Pusat   
3                      31.0                    Menteng         Jakarta Pusat   
4                      31.0                    Menteng         Jakarta Pusat   

        properties.nama_provinsi  
0  DAERAH KHUSUS IBUKOTA JAKARTA  
1  DAERAH KHUSUS IBUKOTA JAKARTA  
2  DAERAH KHUSUS IBUKOTA JAKARTA  
3  DAERAH KHUSUS IBUKOTA JAKARTA  
4  DAERAH KHUSUS IBUKOTA JAKARTA  

[5 rows x 22 columns]

The dataset now has information from both `hospital` and `district` data frame using `kode_kecamatan` as the reference index. This can be a very handy tools for collecting unstructured data into a tabular one.

#### Knowledge Check

If we inspect how many each district has a general hospital we can do the following:

```
df_merged['properties.nama_kecamatan'].value_counts()
```

That will give you a simple frequency table that shows how many each of the district occurs in the dataset. However, we can not know for sure if there are some of the district that has no hospital at all. Recall what we learned about Categorical data! In the following chunk do some initial data transformation using `pd.Categorical` for `district` data frame. After the data types has been changed check it with `district.dtypes` to make sure it has been stored in the correct types:

In [126]:
# Change to categoricla type here


# Checking data types
district.dtypes

properties.kode_kecamatan     int64
properties.kode_kota          int64
properties.kode_provinsi      int64
properties.nama_kecamatan    object
properties.nama_kota         object
properties.nama_provinsi     object
dtype: object

Last, merge `hospital` and `district` and do the `value_counts` on `nama_kecamatan` column. See how does it different with the previous one before you change the type!

In [127]:
# Merge the data frame and count the value



# Learn-by-Building
## Graded Assignment 4
### SQL and Exploratory Data Analysis

The following learn-by-building exercise will guide you through the process of building out a simple data analysis. This module is considerably more difficult than similar exercise blocks in the past, but it sure is a lot more rewarding!

Let's try by first constructing a DataFrame using the `read_sql_query()` method that we've grown familiar to. We want to develop a simple sales visualization report of our top 5 key markets (`Country` column in `customers`) ranked by Sales (`Total` column in `invoices`). 

We also want to identify our top 5 customers by name (`FirstName`, `LastName`) in the report.

Last but not least, we want the report to include a day-of-week analysis on sales performance, and for that we will need the `InvoiceDate` column. 

> **Hint 1**: `pandas` has built-in methods of extracting the name of day in a week. We've seen this in Part 2 of this specialization (**Working with Datetime chapter**). An example usage is:
>
> `x['InvoiceDOW'] = x['InvoiceDate'].dt.weekday_name`
>
>  **Hint 2**: In `read_sql_query`, you can use the `parse_dates='InvoiceDate'` argument to have the specified column parsed as date, saving you from a `to_datetime()` conversion

In [195]:
## Your code below


## -- Solution code

1. Which of the following customers are among the top 5 customers ranked by their respective total (`Total`)? Choose all that apply.
    - [ ] Victor Stevens
    - [ ] Hugh O'Reilly
    - [ ] Puja Srivastava
    - [ ] Ellie Sullivan

2. Which of the following countries are among the top 5 grossing ranked by their respective total (`Total`)? Choose all that apply.
    - [ ] United Kingdom
    - [ ] Germany
    - [ ] Portugal
    - [ ] India
    
3. How many sales transactions (or records, or rows) were from the top 5 countries?
    - Answer:

In [245]:
## Your code below


## -- Solution code

### Working with API and JSON

Now using the same open API we have tried. Access the following url:

```
http://api.jakarta.go.id/v1/emergency/petugaspemadam/
```

Transform the JSON into a dataframe. Don't forget to perform data transformation as needed!



In [148]:
## Your code below


## -- Solution code

4. Which of the city has the least amount of fire officer dispatched in that particular city?

    - [ ] Jakarta Timur
    - [ ] Jakarta Utara
    - [ ] Jakarta Selatan
    - [ ] Jakarta Pusat
    
5. Which of the following fire officer that is not listed as the head of a platoon (Kepala Pleton)?

    - [ ] Ali Basuki
    - [ ] Tri Wiyanta
    - [ ] Hanafi Yahya
    - [ ] Ahmadi

6. Who is the fire officer that is recorded to last logged in into the system?

    - [ ] Wardiyono, S.E.
    - [ ] Paryanto
    - [ ] Dwi Ratna Swadharma
    - [ ] Usman, Sp.Pd.
    
> **Hint**: To find out the last logged in person transform the column into a date time and do subsetting based on the `.max()` value  